## Set 5
## 3. Word2Vec \*\*Principles**

#### Preparation

In [2]:
# download the helper function
!wget -O P3CHelpers.py https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/P3CHelpers.py

--2022-02-16 22:33:44--  https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/P3CHelpers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4939 (4.8K) [text/plain]
Saving to: ‘P3CHelpers.py’

P3CHelpers.py       100%[===================>]   4.82K  --.-KB/s    in 0s      

2022-02-16 22:33:44 (63.3 MB/s) - ‘P3CHelpers.py’ saved [4939/4939]



In [3]:
# download the dataset
!wget -O dr_seuss.txt https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/data/dr_seuss.txt

--2022-02-16 22:33:45--  https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set5/data/dr_seuss.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8810 (8.6K) [text/plain]
Saving to: ‘dr_seuss.txt’

dr_seuss.txt        100%[===================>]   8.60K  --.-KB/s    in 0s      

2022-02-16 22:33:45 (69.4 MB/s) - ‘dr_seuss.txt’ saved [8810/8810]



In [4]:
import numpy as np
from P3CHelpers import *
import torch
import torch.nn as nn
import torch.optim as optim

#### Problem D: 
Fill in the generate_traindata and find_most_similar_pairs functions.

In [60]:
def get_word_repr(word_to_index, word):
    """
    Returns one-hot-encoded feature representation of the specified word given
    a dictionary mapping words to their one-hot-encoded index.

    Arguments:
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        word:          Word whose feature representation we wish to compute.

    Returns:
        feature_representation:     Feature representation of the passed-in word.
    """
    unique_words = word_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[word_to_index[word]] = 1
    return feature_representation    

def generate_traindata(word_list, word_to_index, window_size=4):
    """
    Generates training data for Skipgram model.

    Arguments:
        word_list:     Sequential list of words (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window. Defaults to 2 
                       (use the default value when running your code).

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training 
                              points (one-hot-encoded vectors) and their corresponding output_word
                              (also one-hot-encoded vectors)

    """
    trainX = []
    trainY = []

    for i in range(len(word_list)):
      pX = get_word_repr(word_to_index, word_list[i])
      for j in range(max(0,i-window_size), min(len(word_list), i+window_size+1)):
        pY = get_word_repr(word_to_index, word_list[j])
        if i != j:
          trainX.append(pX)
          trainY.append(pY)
    return np.array(trainX), np.array(trainY)

In [179]:
def find_most_similar_pairs(filename, num_latent_factors):
    """
    Find the most similar pairs from the word embeddings computed from
    a body of text
    
    Arguments:
        filename:           Text file to read and train embeddings from
        num_latent_factors: The number of latent factors / the size of the embedding
    """
    # Load in a list of words from the specified file; remove non-alphanumeric characters
    # and make all chars lowercase.
    sample_text = load_word_list(filename)

    # Create word dictionary
    word_to_index = generate_onehot_dict(sample_text)
    print("Textfile contains %s unique words"%len(word_to_index))
    # Create training data
    trainX, trainY = generate_traindata(sample_text, word_to_index)

    # vocab_size = number of unique words in our text file. Will be useful 
    # when adding layers to your neural network
    vocab_size = len(word_to_index)

    model = nn.Sequential(
        nn.Linear(vocab_size, 10),
        nn.Linear(10, vocab_size),
        nn.Softmax()
    )
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    loss_fn = nn.CrossEntropyLoss()
    train_dataset = torch.from_numpy(np.array([trainX,trainY])).type(torch.FloatTensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    model.train()

    for epoch in range(20):
        for batch_idx, (data, target) in enumerate(train_loader):
            # Erase accumulated gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(data)

            # Calculate loss
            loss = loss_fn(output, target)

            # Backward pass
            loss.backward()
            
            # Weight update
            optimizer.step()

        # Track loss each epoch
        print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

    model.eval()
    [print(p[0],p[1].shape) for p in model.named_parameters()]
    
    # set weights variable below
    weights = model.get_parameter('1.weight').detach()
    
    # Find and print most similar pairs
    similar_pairs = most_similar_pairs(weights, word_to_index)
    for pair in similar_pairs[:30]:
        print(pair)

### Problem E-H:
Run your model on drseuss.txt and answer questions from E through H.

In [180]:
find_most_similar_pairs('dr_seuss.txt', 10)

Textfile contains 308 unique words


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 1  Loss: 5.7301
Train Epoch: 2  Loss: 5.7300
Train Epoch: 3  Loss: 5.7300
Train Epoch: 4  Loss: 5.7299
Train Epoch: 5  Loss: 5.7298
Train Epoch: 6  Loss: 5.7298
Train Epoch: 7  Loss: 5.7297
Train Epoch: 8  Loss: 5.7296
Train Epoch: 9  Loss: 5.7295
Train Epoch: 10  Loss: 5.7293
Train Epoch: 11  Loss: 5.7292
Train Epoch: 12  Loss: 5.7290
Train Epoch: 13  Loss: 5.7288
Train Epoch: 14  Loss: 5.7285
Train Epoch: 15  Loss: 5.7282
Train Epoch: 16  Loss: 5.7279
Train Epoch: 17  Loss: 5.7275
Train Epoch: 18  Loss: 5.7271
Train Epoch: 19  Loss: 5.7266
Train Epoch: 20  Loss: 5.7261
0.weight torch.Size([10, 308])
0.bias torch.Size([10])
1.weight torch.Size([308, 10])
1.bias torch.Size([308])
Pair(up, cats), Similarity: 0.97992325
Pair(cats, up), Similarity: 0.97992325
Pair(it, ring), Similarity: 0.97332704
Pair(ring, it), Similarity: 0.97332704
Pair(wump, light), Similarity: 0.96600795
Pair(light, wump), Similarity: 0.96600795
Pair(about, top), Similarity: 0.96423125
Pair(top, about),